In [16]:
#Functional API fusion model
import keras
import tensorflow
#tensorflow.debugging.set_log_device_placement(True)
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Conv3D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Concatenate
from keras.layers import Input
from tensorflow.keras import optimizers

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import datetime
from sklearn.model_selection import KFold
import pickle
from sklearn.utils import shuffle
import sys
np.set_printoptions(threshold=1000)

In [17]:
gpus = tensorflow.config.list_physical_devices('GPU')


strategy = tensorflow.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])#devices=["/gpu:0", "/gpu:1"]

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [18]:
load = np.load('data/RAVDESS_basic.npz',allow_pickle=True)
audio = load['audio']
video = load['video']
labels = load['labels']
n_splits = 6
print(np.array(video.shape))
print(np.array(audio.shape))
print(np.array(labels.shape))

[1152  115  128  128]
[  1152 110544]
[1152]


In [19]:
#video pre-process
fraction = 5
frames = int(video.shape[1]/fraction) #115
shape = (128,128,frames,1)

print(video.shape)
reshape_videos = []

for i in video:
    single_video = []
    for j in range( i.shape[0] ): #HEREi.shape[0]
        if(  j % fraction == 0 ):
            single_video.append( i[j] )
    reshape_videos.append( single_video )

video = np.array(reshape_videos)
del reshape_videos

print(video.shape)

video_input = (128, 128, frames, 1)
video_shape = (video.shape[0], 128, 128, frames, 1)
video = np.reshape(video, video_shape)

(1152, 115, 128, 128)
(1152, 23, 128, 128)


In [20]:
#audio pre-process
num_rows = 110544 #84672 
num_columns = 1
num_channels = 1

audio_input = (num_rows, num_columns)
audio_shape = (audio.shape[0], num_rows, num_columns)
audio = np.reshape(audio, audio_shape)

In [21]:
#Audio Network
def audio_cnn_1d(input_):
    #Layer 1
    conv1d = Conv1D(padding='same', filters=25, kernel_size=64, strides = 8, input_shape=audio_input)(input_)
    relu = LeakyReLU(alpha=0.2)(conv1d)
    
    #Layer 2
    conv1d = Conv1D(padding='same', filters=25, kernel_size=128, strides = 8)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 3
    conv1d = Conv1D(padding='same', filters=25, kernel_size=256, strides = 8)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 4
    conv1d = Conv1D(padding='same', filters=25, kernel_size=512, strides = 4)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #Layer 5
    conv1d = Conv1D(padding='same', filters=25, kernel_size=1024, strides = 4)(relu)
    batch = BatchNormalization()(conv1d)
    relu = LeakyReLU(alpha=0.2)(relu)
    
    #layer 6
    output = Flatten()(relu)
    
    return output

In [22]:
#video neural network
def video_cnn_3d(input_):
    #layer 1
    conv3d = Conv3D(64, kernel_size=(3,3,3), strides=(2,2,1), input_shape=video_input)(input_)
    relu = LeakyReLU(alpha=0.2)(conv3d)
    
    #layer 2
    conv3d = Conv3D(128, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)
    
    #layer 3
    conv3d = Conv3D(256, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 4
    conv3d = Conv3D(512, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 5
    conv3d = Conv3D(1024, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 6
    conv3d = Conv3D(2048, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME')(relu)
    batch = BatchNormalization()(conv3d)
    relu = LeakyReLU(alpha=0.2)(batch)

    #layer 7
    output = Flatten()(relu)
    return output


In [23]:
#labels = labels - 1
labels = to_categorical(labels)
target_names = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']


In [27]:
#k-Fold training, set for 6 splits

n = 0 #current fold

epochs = 10
function = 'softmax'
name = 'ravdess_' + str(epochs) + '_' + function 
pickle_file = 'pickle_files/' + name + '.pckl'

training_score = []
testing_score = []

f = open(pickle_file, 'wb') #file for saving information/data about each fold


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)

print(kf)

print(audio.shape)
print(labels.shape)

for train_index, test_index in kf.split(audio):
    print('Beginning fold #' + str(n) + '...')
    #print(train_index)
    #print(test_index)
    a_train, a_test = audio[train_index], audio[test_index]
    v_train, v_test = video[train_index], video[test_index]
    l_train, l_test = labels[train_index], labels[test_index]
    
    
    with strategy.scope():
        #fusion layer
        aud = Input(audio_input)
        vid = Input(video_input)

        audio_tensor = audio_cnn_1d(aud)

        video_tensor = video_cnn_3d(vid)

        fusion = Concatenate()([audio_tensor, video_tensor]) #combining tensors
        batch = BatchNormalization()(fusion)
        prediction = Dense(6,activation=function)(batch) #softmax
        twin_net = Model(inputs=[aud,vid], outputs=prediction)

        #Training

        checkpoint_filepath = 'models/' + name
        model_checkpoint_callback = ModelCheckpoint(
            filepath=checkpoint_filepath,
            period=5
            )
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + checkpoint_filepath
        tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        twin_net.compile(loss='categorical_crossentropy', #categorical_crossentropy
                optimizer=optimizers.Adam(learning_rate=0.00001), 
                         metrics=['accuracy'])

        #twin_net.summary()


        twin_net.fit([a_train,v_train], l_train,
                batch_size=30,
                epochs=epochs,
                verbose=1,
                validation_split=0.166,
                callbacks=[model_checkpoint_callback,
                tensorboard_callback])
        
        score = twin_net.evaluate([a_train, v_train], l_train, verbose=0)
        pickle.dump([score, 'training accuracy fold # 7/6/2021' + str(n)], f)
        training_score.append(score)

        score = twin_net.evaluate([a_test, v_test], l_test, verbose=0)
        pickle.dump([score, 'testing accuracy fold #' + str(n)], f)
        testing_score.append(score)

        Y_pred = twin_net.predict([a_test, v_test])
        y_pred = np.argmax(Y_pred, axis=1)
        labels_ = np.argmax(l_test, axis=1)
        pickle.dump(confusion_matrix(labels_, y_pred), f)
        pickle.dump(classification_report(labels_, y_pred), f)
        
        n+=1
        
    break
        
f.close()
np.savez('data/'+name,x=training_score,y=testing_score)





KFold(n_splits=6, random_state=42, shuffle=True)
(1152, 110544, 1)
(1152, 6)
Beginning fold #0...
Epoch 1/10
27/27 [==============================] - 21s 512ms/step - loss: 2.1048 - accuracy: 0.4200 - val_loss: 1.8284 - val_accuracy: 0.1562
Epoch 2/10
27/27 [==============================] - 9s 349ms/step - loss: 0.3761 - accuracy: 0.8700 - val_loss: 1.8597 - val_accuracy: 0.1750
Epoch 3/10
27/27 [==============================] - 9s 345ms/step - loss: 0.0909 - accuracy: 0.9800 - val_loss: 1.9448 - val_accuracy: 0.1937
Epoch 4/10
27/27 [==============================] - 9s 343ms/step - loss: 0.0417 - accuracy: 0.9950 - val_loss: 2.0042 - val_accuracy: 0.2313
Epoch 5/10
27/27 [==============================] - 9s 345ms/step - loss: 0.0216 - accuracy: 0.9950 - val_loss: 2.0348 - val_accuracy: 0.2688
INFO:tensorflow:Assets written to: models/ravdess_10_softmax/assets
Epoch 6/10
27/27 [==============================] - 9s 342ms/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 2.0365 - va

In [29]:
load = np.load('data/'+name+'.npz')
training_score = load['x']
testing_score = load['y']

print(training_score,testing_score)
#print('testing_score loss:', np.mean(testing_score[0]))
#print('testing_score accuracy:',np.mean(testing_score[1]))



f.close()

with (open(pickle_file, "rb")) as openfile:
    while True:
        try:
            print(pickle.load(openfile))
        except EOFError:
            break

[[0.47655532 0.88645834]] [[0.59111792 0.828125  ]]
[[0.47655531764030457, 0.8864583373069763], 'training accuracy fold # 7/6/20210']
[[0.5911179184913635, 0.828125], 'testing accuracy fold #0']
[[31  0  1  0  0  0]
 [ 2 26  1  1  4  0]
 [ 4  0 24  0  4  0]
 [ 0  0  1 22  3  2]
 [ 0  0  1  0 30  0]
 [ 2  0  3  1  3 26]]
              precision    recall  f1-score   support

           0       0.79      0.97      0.87        32
           1       1.00      0.76      0.87        34
           2       0.77      0.75      0.76        32
           3       0.92      0.79      0.85        28
           4       0.68      0.97      0.80        31
           5       0.93      0.74      0.83        35

    accuracy                           0.83       192
   macro avg       0.85      0.83      0.83       192
weighted avg       0.85      0.83      0.83       192



INFO:tensorflow:Assets written to: models/ravdess_30_sigmoid/assets
[[0.3629540205001831, 0.8600000143051147], 'training accuracy fold #0']
[[0.5187535285949707, 0.7833333611488342], 'testing accuracy fold #0']
[[15  0  0  1  0  1  0  0]
 [ 2 32  0  2  0  2  0  0]
 [ 0  0 25  2  0  2  3  0]
 [ 2  0  0 19  5  3  1  0]
 [ 0  1  0  2 27  1  0  0]
 [ 2  2  0  0  2 20  2  0]
 [ 0  0  0  0  2  3 25  0]
 [ 0  0  4  2  0  1  2 25]]
              precision    recall  f1-score   support

           1       0.71      0.88      0.79        17
           2       0.91      0.84      0.88        38
           3       0.86      0.78      0.82        32
           4       0.68      0.63      0.66        30
           5       0.75      0.87      0.81        31
           6       0.61      0.71      0.66        28
           7       0.76      0.83      0.79        30
           8       1.00      0.74      0.85        34

    accuracy                           0.78       240
   macro avg       0.79      0.79      0.78       240
weighted avg       0.80      0.78      0.79       240

[[0.3796142041683197, 0.8700000047683716], 'training accuracy fold #1']
[[0.5539751648902893, 0.7708333134651184], 'testing accuracy fold #1']
[[19  0  0  0  1  0  0  0]
 [ 0 20  0  0  0  3  0  0]
 [ 0  1 29  0  4  1  1  2]
 [ 2  0  0 26  2  3  0  2]
 [ 0  1  0  0 24  5  0  0]
 [ 1  0  0  1  3 23  3  1]
 [ 0  0  0  0  5  5 27  0]
 [ 0  1  0  0  3  2  2 17]]
              precision    recall  f1-score   support

           1       0.86      0.95      0.90        20
           2       0.87      0.87      0.87        23
           3       1.00      0.76      0.87        38
           4       0.96      0.74      0.84        35
           5       0.57      0.80      0.67        30
           6       0.55      0.72      0.62        32
           7       0.82      0.73      0.77        37
           8       0.77      0.68      0.72        25

    accuracy                           0.77       240
   macro avg       0.80      0.78      0.78       240
weighted avg       0.81      0.77      0.78       240

[[0.37501320242881775, 0.8641666769981384], 'training accuracy fold #2']
[[0.47459012269973755, 0.7916666865348816], 'testing accuracy fold #2']
[[12  0  0  0  1  1  0  0]
 [ 2 28  2  0  0  2  0  1]
 [ 2  0 26  0  2  2  0  1]
 [ 0  0  0 23  3  0  2  1]
 [ 0  2  0  3 25  0  0  0]
 [ 0  0  0  2  4 17  2  0]
 [ 0  0  0  2  2  2 27  2]
 [ 0  0  0  1  3  2  1 32]]
              precision    recall  f1-score   support

           1       0.75      0.86      0.80        14
           2       0.93      0.80      0.86        35
           3       0.93      0.79      0.85        33
           4       0.74      0.79      0.77        29
           5       0.62      0.83      0.71        30
           6       0.65      0.68      0.67        25
           7       0.84      0.77      0.81        35
           8       0.86      0.82      0.84        39

    accuracy                           0.79       240
   macro avg       0.79      0.79      0.79       240
weighted avg       0.81      0.79      0.80       240

[[0.3411281406879425, 0.8633333444595337], 'training accuracy fold #3']
[[0.5509750247001648, 0.7791666388511658], 'testing accuracy fold #3']
[[ 8  2  0  1  0  0  0  1]
 [ 0 23  0  4  0  1  0  1]
 [ 0  0 29  1  3  1  0  2]
 [ 0  2  0 30  5  4  0  2]
 [ 0  1  2  1 32  3  0  0]
 [ 0  1  0  0  4 24  0  0]
 [ 0  0  0  2  2  0 15  0]
 [ 0  3  0  1  2  0  1 26]]
              precision    recall  f1-score   support

           1       1.00      0.67      0.80        12
           2       0.72      0.79      0.75        29
           3       0.94      0.81      0.87        36
           4       0.75      0.70      0.72        43
           5       0.67      0.82      0.74        39
           6       0.73      0.83      0.77        29
           7       0.94      0.79      0.86        19
           8       0.81      0.79      0.80        33

    accuracy                           0.78       240
   macro avg       0.82      0.77      0.79       240
weighted avg       0.79      0.78      0.78       240

[[0.36666935682296753, 0.8608333468437195], 'training accuracy fold #4']
[[0.4919424057006836, 0.800000011920929], 'testing accuracy fold #4']
[[11  1  0  1  1  0  0  0]
 [ 4 34  0  2  5  1  0  0]
 [ 0  2 15  2  1  1  0  0]
 [ 0  0  0 20  2  2  0  0]
 [ 1  0  0  2 20  5  0  0]
 [ 0  0  0  0  2 34  0  0]
 [ 0  0  0  4  3  1 31  0]
 [ 0  0  0  0  2  1  2 27]]
              precision    recall  f1-score   support

           1       0.69      0.79      0.73        14
           2       0.92      0.74      0.82        46
           3       1.00      0.71      0.83        21
           4       0.65      0.83      0.73        24
           5       0.56      0.71      0.63        28
           6       0.76      0.94      0.84        36
           7       0.94      0.79      0.86        39
           8       1.00      0.84      0.92        32

    accuracy                           0.80       240
   macro avg       0.81      0.80      0.79       240
weighted avg       0.83      0.80      0.81       240

[[0.3825329542160034, 0.8700000047683716], 'training accuracy fold #5']
[[0.580239474773407, 0.7916666865348816], 'testing accuracy fold #5']
[[15  0  0  1  2  1  0  0]
 [ 0 18  0  0  1  2  0  0]
 [ 0  0 26  0  4  2  0  0]
 [ 0  0  0 23  6  0  0  2]
 [ 0  0  0  0 33  1  0  0]
 [ 0  3  0  2  5 32  0  0]
 [ 1  0  0  2  2  4 22  1]
 [ 0  0  0  0  6  2  0 21]]
              precision    recall  f1-score   support

           1       0.94      0.79      0.86        19
           2       0.86      0.86      0.86        21
           3       1.00      0.81      0.90        32
           4       0.82      0.74      0.78        31
           5       0.56      0.97      0.71        34
           6       0.73      0.76      0.74        42
           7       1.00      0.69      0.81        32
           8       0.88      0.72      0.79        29

    accuracy                           0.79       240
   macro avg       0.85      0.79      0.81       240
weighted avg       0.83      0.79      0.80       240

In [ ]:
with (open(pickle_file, "rb")) as openfile:
    while True:
        try:
            print(pickle.load(openfile))
        except EOFError:
            break



In [ ]:
with strategy.scope():
    #test sigmoid
    model = load_model('models/ravdess_softmax_extended')


    score = model.evaluate([a_train, v_train], l_train, verbose=0)
    print("Training Accuracy: ", score)

    score = model.evaluate([a_test, v_test], l_test, verbose=0)
    print("Testing Accuracy: ", score)


    Y_pred = model.predict([a_test, v_test])
    y_pred = np.argmax(Y_pred, axis=1)
    labels = np.argmax(l_test, axis=1)
    print('Confusion_Matrix')
    print(confusion_matrix(labels, y_pred))

    print('Classification Report')
    print(classification_report(labels, y_pred, target_names=target_names))


300 epoch softmax
Training Accuracy:  [0.3390514552593231, 0.9666666388511658]
Testing Accuracy:  [2.35433292388916, 0.7958333492279053]
Confusion_Matrix
[[17  0  0  0  0  0  0  0]
 [ 3 31  0  0  2  0  1  1]
 [ 0  0 23  4  0  1  2  2]
 [ 2  0  2 22  2  0  1  1]
 [ 0  0  0  3 27  0  0  1]
 [ 3  2  0  0  2 18  3  0]
 [ 0  0  0  1  0  2 27  0]
 [ 0  1  3  1  0  0  3 26]]
Classification Report
              precision    recall  f1-score   support

     neutral       0.68      1.00      0.81        17
        calm       0.91      0.82      0.86        38
       happy       0.82      0.72      0.77        32
         sad       0.71      0.73      0.72        30
       angry       0.82      0.87      0.84        31
     fearful       0.86      0.64      0.73        28
     disgust       0.73      0.90      0.81        30
   surprised       0.84      0.76      0.80        34

    accuracy                           0.80       240
   macro avg       0.80      0.81      0.79       240
weighted avg       0.81      0.80      0.79       240


Training Accuracy:  [0.3390514552593231, 0.9666666388511658]
Testing Accuracy:  [2.35433292388916, 0.7958333492279053]
Confusion_Matrix
[[17  0  0  0  0  0  0  0]
 [ 3 31  0  0  2  0  1  1]
 [ 0  0 23  4  0  1  2  2]
 [ 2  0  2 22  2  0  1  1]
 [ 0  0  0  3 27  0  0  1]
 [ 3  2  0  0  2 18  3  0]
 [ 0  0  0  1  0  2 27  0]
 [ 0  1  3  1  0  0  3 26]]
Classification Report
              precision    recall  f1-score   support

     neutral       0.68      1.00      0.81        17
        calm       0.91      0.82      0.86        38
       happy       0.82      0.72      0.77        32
         sad       0.71      0.73      0.72        30
       angry       0.82      0.87      0.84        31
     fearful       0.86      0.64      0.73        28
     disgust       0.73      0.90      0.81        30
   surprised       0.84      0.76      0.80        34

    accuracy                           0.80       240
   macro avg       0.80      0.81      0.79       240
weighted avg       0.81      0.80      0.79       240

sigmoid 300 ravdess
Training Accuracy:  [0.019113920629024506, 0.9758333563804626]
Testing Accuracy:  [0.13955293595790863, 0.8541666865348816]
Confusion_Matrix
[[17  0  0  0  0  0  0  0]
 [ 0 35  1  1  0  0  0  1]
 [ 0  0 29  0  0  0  2  1]
 [ 0  2  2 22  2  0  1  1]
 [ 0  0  0  2 27  0  1  1]
 [ 0  1  0  1  2 21  3  0]
 [ 0  0  0  1  1  2 26  0]
 [ 0  0  4  0  0  0  2 28]]
Classification Report
              precision    recall  f1-score   support

     neutral       1.00      1.00      1.00        17
        calm       0.92      0.92      0.92        38
       happy       0.81      0.91      0.85        32
         sad       0.81      0.73      0.77        30
       angry       0.84      0.87      0.86        31
     fearful       0.91      0.75      0.82        28
     disgust       0.74      0.87      0.80        30
   surprised       0.88      0.82      0.85        34

    accuracy                           0.85       240
   macro avg       0.86      0.86      0.86       240
weighted avg       0.86      0.85      0.85       240



sigmoid
Training Accuracy:  [0.15417054295539856, 0.9775000214576721]
Testing Accuracy:  [0.3786163330078125, 0.824999988079071]
Confusion_Matrix
[[17  0  0  0  0  0  0  0]
 [ 0 35  1  0  0  1  0  1]
 [ 0  4 24  0  0  2  2  0]
 [ 2  0  2 22  2  0  1  1]
 [ 0  0  0  1 29  0  0  1]
 [ 2  0  2  0  0 20  4  0]
 [ 0  0  1  1  0  2 26  0]
 [ 0  0  5  0  0  1  3 25]]
Classification Report
              precision    recall  f1-score   support

     neutral       0.81      1.00      0.89        17
        calm       0.90      0.92      0.91        38
       happy       0.69      0.75      0.72        32
         sad       0.92      0.73      0.81        30
       angry       0.94      0.94      0.94        31
     fearful       0.77      0.71      0.74        28
     disgust       0.72      0.87      0.79        30
   surprised       0.89      0.74      0.81        34

    accuracy                           0.82       240
   macro avg       0.83      0.83      0.83       240
weighted avg       0.83      0.82      0.82       240


In [ ]:
#test softmax
model = load_model('models/twin_net_softmax')


score = model.evaluate([a_train, v_train], l_train, verbose=0)
print("Training Accuracy: ", score)

score = model.evaluate([a_test, v_test], l_test, verbose=0)
print("Testing Accuracy: ", score)


Y_pred = model.predict([a_test, v_test])
y_pred = np.argmax(Y_pred, axis=1)
labels = np.argmax(l_test, axis=1)
print('Confusion_Matrix')
print(confusion_matrix(labels, y_pred))
